In [ ]:
import numpy
import mxnet as mx

#Faz o download do dataset mnist
mnist = mx.test_utils.get_mnist()

# Fix the seed
# Seta o valor aleatório inicial
mx.random.seed(42)

# Define o contexto computacional, se a GPU estiver disponivel vai executar nela pq é mais rápido, senão na CPU
ctx_comp = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()


In [ ]:
#lote de 100 "neurônios"
tam_lote= 100

#referencia= https://mxnet.incubator.apache.org/api/python/io/io.html
#http://mxnet.incubator.apache.org/test/versions/0.10/api/python/ndarray.html#module-mxnet.ndarray
#procurar youtube data interator
#The NDArray API, defined in the ndarray (or simply nd) package, provides imperative tensor operations on CPU/GPU. 
#An NDArray represents a multi-dimensional, fixed-size homogenous array.

#(tam_lote, num_channels, width, height)

#inicializou e embaralhou o vetor de imagens de teste e seus labels(onde diz qual é o número que está na imagem) 
dados_treino = mx.io.NDArrayIter(mnist['train_data'], mnist['train_label'], tam_lote, shuffle=True)
print(dados_treino.provide_data)
print(dados_treino.provide_label)

#inicializou o vetor de teste
dados_teste = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], tam_lote)
print(dados_teste.provide_data)
print(dados_teste.provide_label)

In [ ]:
# MULTILAYER PERCEPTRON (MPL)

# criando um local reservado para o dado de entrada
dado_entrada = mx.sym.var('data')

# Flatten the data from 4-D shape into 2-D (tam_lote, num_channel*width*height)
dato_entrada = mx.sym.flatten(data=data)

In [ ]:
# The first fully-connected layer and the corresponding activation function
#seta o numero de neoronios da camada 1 
camada1  = mx.sym.FullyConnected(data=dado_entrada, num_hidden=128)
#Diz a activation function usada.
resultado1 = mx.sym.Activation(data=camada1, act_type="relu")

# The second fully-connected layer and the corresponding activation function
camada2  = mx.sym.FullyConnected(data=resultado1, num_hidden = 64)
resultado2 = mx.sym.Activation(data=camada2, act_type="relu")

In [ ]:
# MNIST tem 10 classes (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

# O dado da camada 3 é a camada  2 com a função de ativação(RELU) aplicada
camada3  = mx.sym.FullyConnected(data=camada2, num_hidden=10)

# Softmax com cross entropy loss
# Softmax= Distribuição de probalidade entre 0 e 1
# Cross-entropy= Mede o número médio de bits necessários para identificar um evento 
mlp  = mx.sym.SoftmaxOutput(data=camada3, name='softmax')

![png](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/mlp_mnist.png)

**Figure 1:** MLP network architecture for MNIST.

In [ ]:
# TREINAMENTO

# Stochastic gradient descent (SGD)= busca minimizar a loss function(normalmente a diferença entre a saída prevista e a real), 
# escolhendo os melhores parâmetros de entrada dos que tem disponível.

# Cria um módulo de treino no contexto computacional atual(GPU ou CPU)
mlp_model = mx.mod.Module(symbol=mlp, context=ctx)

# Parêmtros do modelo
mlp_model.fit(dados_treino, # Dados de treino
              
     eval_data=dados_teste, # Dados para avaliação
              
     optimizer='sgd', # Usa o método de otimização SGD
              
     optimizer_params={'learning_rate':0.6}, # Controla o tamanho da passo que o otimizador toma em busca de uma solução.
     # Este parâmetro informa ao otimizador até onde mover os pesos na direção do gradiente para um mini-lote.
              
      eval_metric='acc', # Mostra a precisão do treino
              
      batch_end_callback = mx.callback.Speedometer(tam_lote,100), # Mostra progresso de treino para cada 100 lotes de 100 imagens
              
      num_epoch=10) # Passa 10 vezes o dataset inteiro

In [ ]:
teste = mx.io.NDArrayIter(mnist['test_data'], None, tam_lote) # Vetor setado com as imagens, sem os labels para teste

prob = mlp_model.predict(teste) # Calcula a probabilidade de cada imagem das 10000, ser de cada uma das 10 classes

assert prob.shape == (10000, 10) # verificação em tempo de execução de uma condição qualquer. Se a condição não for verdadeira, 
                                 # uma exceção AssertionError acontece e o programa para

In [ ]:
teste = mx.io.NDArrayIter(mnist['test_data'], mnist['test_label'], tam_lote)

# Previsão da precisão do modelo mlp
precisao = mx.metric.Accuracy()
mlp_model.score(teste, precisao)
print(precisao)
assert precisao.get()[1] > 0.96, "A precisão alcançada (%f) é menor do que a esperada (0.96)" % precisao.get()[1]